In [2]:
import random
import json
import shutil
from pathlib import Path
import pickle
from collections import defaultdict
import os
from tqdm import tqdm

In [2]:

with open('../src/graph_embeddings/image_ge_embeddings_cos_GE_512_d512_l0.0001_105_05_2.pkl', 'rb') as f:
    image_dict = pickle.load(f)

In [26]:
def split_test_sets(image_folder, json_path, output_folder, seed=42):
    """
    Split images into test_query and test_gallery ensuring:
    1. Two images from the same patent go to test_query
    2. Rest of the images from same patent go to test_gallery
    3. Maintains information about Locarno classification for evaluation
    """
    # Set random seed for reproducibility
    random.seed(seed)

    # Load JSON metadata
    with open(json_path, "r") as f:
        metadata = json.load(f)

    # Group images by patent ID
    patent_groups = defaultdict(list)
    for item in metadata:
        pat=item["patentID"]
        
        patent_groups[item["patentID"]].append(item)

    # Create output directories
    test_query_dir = Path(output_folder) / "test_query"
    test_gallery_dir = Path(output_folder) / "test_gallery"
    #test_query_dir.mkdir(parents=True, exist_ok=True)
    #test_gallery_dir.mkdir(parents=True, exist_ok=True)

    # Prepare metadata for split sets
    query_metadata = []
    gallery_metadata = []
    print(patent_groups)
    '''
    # Split images
    for patent_id, items in patent_groups.items():
        if len(items) >= 3:  # Only process patents with at least 3 images
            # Randomly select 2 images for query
            query_items = random.sample(items, 2)
            # Rest go to gallery
            gallery_items = [item for item in items if item not in query_items]

            # Copy query images and save metadata
            for item in query_items:
                #src_path = Path(image_folder) / item["subfigure_file"]
                #dst_path = test_query_dir / item["subfigure_file"]
                #shutil.copy2(src_path, dst_path)
                query_metadata.append(item)

            # Copy gallery images and save metadata
            for item in gallery_items:
                #src_path = Path(image_folder) / item["subfigure_file"]
                #dst_path = test_gallery_dir / item["subfigure_file"]
                #shutil.copy2(src_path, dst_path)
                gallery_metadata.append(item)

    # Save split metadata
    with open(Path(output_folder) / "test_query_metadata_2.json", "w") as f:
        json.dump(query_metadata, f, indent=2)
    
    with open(Path(output_folder) / "test_gallery_metadata_2.json", "w") as f:
        json.dump(gallery_metadata, f, indent=2)

    # Print statistics
    print(f"Split complete:")
    print(f"Number of query images: {len(query_metadata)}")
    print(f"Number of gallery images: {len(gallery_metadata)}")

    # Create ground truth dictionary for evaluation
    ground_truth = create_ground_truth(query_metadata, gallery_metadata)

    return ground_truth
    '''

def create_ground_truth(query_metadata,image_dict):
    """
    Create ground truth dictionary for evaluation:
    {
        query_image_name: {
            'patent_positives': [list of gallery images from same patent],
            'cpc_positives': [list of gallery images from same Locarno class]
        }
    }
    """
    ground_truth = {}
    
    
    for query_item in tqdm(query_metadata):
        path= query_item["subfigure_file"]
        date_str = Path(path).name.split('-')[1]
        year = int(date_str[:4])
        month = int(date_str[4:6])
        day = int(date_str[6:8])
        
        
        # Check if the date is before May
        if  month <= 5 and path in list(image_dict.keys()):
            positives = {"patent_positives": [], "cpc_positives": []}

            for gallery_item in query_metadata:
                path= gallery_item["subfigure_file"]
                date_str = Path(path).name.split('-')[1]
                year = int(date_str[:4])
                month = int(date_str[4:6])
                day = int(date_str[6:8])
                
                if  month <= 5 and path in list(image_dict.keys()):
                    
                    # Check patent positives
                    if gallery_item["patentID"] == query_item["patentID"]:
                        positives["patent_positives"].append(gallery_item["subfigure_file"])

                    # Check Locarno positives
                    if (
                        gallery_item["cpc"]
                        == query_item["cpc"]
                    ):
                        positives["cpc_positives"].append(gallery_item["subfigure_file"])
        
            ground_truth[query_item["subfigure_file"]] = positives

    # Save ground truth
    with open("../data/2018/ground_truth_2018_cpc.json", "w") as f:
        json.dump(ground_truth, f, indent=2)

    return ground_truth


# Example usage
#if __name__ == "__main__":
    image_folder = "../data/2018"
    json_path = "../data/2018/design2018_cpc.json"
    output_folder = "../data/2018"
    

    #ground_truth = split_test_sets(image_folder, json_path, output_folder)

In [47]:
with open('../data/2018/design2018_cpc.json', 'r') as f:
    query_metadata = json.load(f)
with open('../data/2018/test_gallery_metadata.json', 'r') as f:
    gallery_metadata = json.load(f)

In [34]:
ground_truth = create_ground_truth(dicto,image_dict)

  0%|          | 5/22964 [00:38<48:54:15,  7.67s/it]


KeyboardInterrupt: 

In [32]:
def split_test_sets(image_folder, json_path, output_folder,image_dict, seed=42):
    """
    Split images into test_query and test_gallery ensuring:
    1. Two images from the same patent go to test_query
    2. Rest of the images from same patent go to test_gallery
    3. Maintains information about Locarno classification for evaluation
    """
    # Set random seed for reproducibility
    random.seed(seed)

    # Load JSON metadata
    with open(json_path, "r") as f:
        metadata = json.load(f)

    # Group images by patent ID
    patent_groups = []
    for item in metadata:
        pat=item["patentID"]
        if int(pat[15:17]) <= 5  and item["subfigure_file"] in list(image_dict.keys()):
            patent_groups.append(item)

    # Create output directories
    test_query_dir = Path(output_folder) / "test_query"
    test_gallery_dir = Path(output_folder) / "test_gallery"
    #test_query_dir.mkdir(parents=True, exist_ok=True)
    #test_gallery_dir.mkdir(parents=True, exist_ok=True)

    # Prepare metadata for split sets
    query_metadata = []
    gallery_metadata = []
    
   
    # Split images
    '''
    for patent_id, items in patent_groups.items():
        if patent_id in 
        if len(items) >= 3:  # Only process patents with at least 3 images
            # Randomly select 2 images for query
            query_items = random.sample(items, 2)
            # Rest go to gallery
            gallery_items = [item for item in items if item not in query_items]

            # Copy query images and save metadata
            for item in query_items:
                #src_path = Path(image_folder) / item["subfigure_file"]
                #dst_path = test_query_dir / item["subfigure_file"]
                #shutil.copy2(src_path, dst_path)
                query_metadata.append(item)

            # Copy gallery images and save metadata
            for item in gallery_items:
                #src_path = Path(image_folder) / item["subfigure_file"]
                #dst_path = test_gallery_dir / item["subfigure_file"]
                #shutil.copy2(src_path, dst_path)
                gallery_metadata.append(item)

    # Save split metadata
    with open(Path(output_folder) / "test_query_metadata_2.json", "w") as f:
        json.dump(query_metadata, f, indent=2)
    
    with open(Path(output_folder) / "test_gallery_metadata_2.json", "w") as f:
        json.dump(gallery_metadata, f, indent=2)

    # Print statistics
    print(f"Split complete:")
    print(f"Number of query images: {len(query_metadata)}")
    print(f"Number of gallery images: {len(gallery_metadata)}")

    # Create ground truth dictionary for evaluation
    ground_truth = create_ground_truth(query_metadata, gallery_metadata)

    return ground_truth
    '''
    return patent_groups
image_folder = "../data/2018"
json_path = "../data/2018/design2018_cpc.json"
output_folder = "../data/2018"
dicto=split_test_sets(image_folder, json_path, output_folder,image_dict, seed=42)

In [3]:
# Load the figure_to_row mapping from the pickle file
with open('image_index_2019.pkl', 'rb') as f:
    figure_to_row = pickle.load(f)

In [4]:
figure_to_row

{'USD0836880-20190101-D00001_1.png': 0,
 'USD0836884-20190101-D00016_16.png': 1,
 'USD0836893-20190101-D00004_4.png': 2,
 'USD0836924-20190101-D00003_3.png': 3,
 'USD0836926-20190101-D00006_6.png': 4,
 'USD0836942-20190101-D00007_7.png': 5,
 'USD0836961-20190101-D00002_2.png': 6,
 'USD0836974-20190101-D00016_16.png': 7,
 'USD0836983-20190101-D00016_18.png': 8,
 'USD0837004-20190101-D00002_6.png': 9,
 'USD0837011-20190101-D00006_6.png': 10,
 'USD0837012-20190101-D00005_5.png': 11,
 'USD0837027-20190101-D00003_9.png': 12,
 'USD0837030-20190101-D00005_5.png': 13,
 'USD0837045-20190101-D00004_7.png': 14,
 'USD0837049-20190101-D00003_6.png': 15,
 'USD0837066-20190101-D00001_1.png': 16,
 'USD0837100-20190101-D00004_6.png': 17,
 'USD0837106-20190101-D00003_3.png': 18,
 'USD0837113-20190101-D00002_6.png': 19,
 'USD0837130-20190101-D00006_6.png': 20,
 'USD0837136-20190101-D00003_3.png': 21,
 'USD0837149-20190101-D00003_3.png': 22,
 'USD0837150-20190101-D00004_4.png': 23,
 'USD0837155-20190101-D

In [8]:
test_images=[]
for filename in os.listdir('../data/2019/test_query_comp'):
    test_images.append(filename)

In [9]:
len(test_images)

1953

In [12]:
import json
import numpy as np
import pickle
from scipy import sparse
from tqdm import tqdm
from collections import defaultdict

# Load the adjacency matrix from the specified path
adj_matrix_path = "../data/2018/graph/combined_adj_query_2019.npz"
combined_adj = sparse.load_npz(adj_matrix_path)

# Convert to CSR format which supports row-wise operations
combined_adj = combined_adj.tocsr()

# Load the figure_to_row mapping from the pickle file
with open('image_index_2019.pkl', 'rb') as f:
    figure_to_row = pickle.load(f)

# Based on your statistics:
num_figures = 95299
num_patents = 11967
num_medium_cpc = 595
num_big_cpc = 150
num_main_cpc = 9


# Calculate column indices for different entity types
figure_cols = list(range(0, num_figures))
patent_cols = list(range(num_figures, num_figures + num_patents))
medium_cpc_cols = list(range(num_figures + num_patents, num_figures + num_patents + num_medium_cpc))

# Step 1: Create mappings
figure_to_patent = {}
patent_to_figures = defaultdict(list)
patent_to_medium_cpc = defaultdict(list)
medium_cpc_to_patents = defaultdict(list)

# First, find connections between patents and medium CPC codes
for patent_idx in tqdm(range(num_patents)):
    # Get the row index for this patent in the adjacency matrix
    patent_row = num_figures + patent_idx
    
    # Get the non-zero entries in this row
    row_data = combined_adj[patent_row, :].nonzero()[1]
    
    # Find medium CPC connections
    for col_idx in row_data:
        if col_idx in medium_cpc_cols:
            cpc_idx = col_idx - (num_figures + num_patents)  # Convert to CPC index
            patent_to_medium_cpc[patent_idx].append(cpc_idx)
            medium_cpc_to_patents[cpc_idx].append(patent_idx)

# Now, find connections between figures and patents
for figure_name, row_idx in tqdm(figure_to_row.items()):
    
    # Convert row_idx to integer if it's not already
    row_idx = int(row_idx)
    
    # Get the non-zero entries in this row
    row_data = combined_adj[row_idx, :].nonzero()[1]
    
    # Find patent connection
    for col_idx in row_data:
        if col_idx in patent_cols:
            patent_idx = col_idx - num_figures  # Convert to patent index
            figure_to_patent[figure_name] = patent_idx
            patent_to_figures[patent_idx].append(figure_name)
            break

# Step 2: Generate ground truth dictionary
ground_truth = {}

for figure_name in tqdm(figure_to_row.keys()):
    if figure_name not in figure_to_patent:
        # Skip figures without patent connections
        continue
    if figure_name not in test_images:
        continue
    
    patent_idx = figure_to_patent[figure_name]
    
    # Find patent positives (figures from same patent)
    patent_positives = [f for f in patent_to_figures[patent_idx] if f != figure_name]
    
    # Find CPC positives (figures from patents that share medium CPC codes)
    cpc_positives = set()
    
    # Get all medium CPC codes for this figure's patent
    medium_cpcs = patent_to_medium_cpc[patent_idx]
    
    # For each medium CPC code, find all patents that have this code
    for cpc_idx in medium_cpcs:
        # For each patent with this CPC code
        for related_patent_idx in medium_cpc_to_patents[cpc_idx]:
            # Add all figures from this patent to CPC positives
            for other_figure in patent_to_figures[related_patent_idx]:
                if other_figure != figure_name:
                    cpc_positives.add(other_figure)
    
    ground_truth[figure_name] = {
        "patent_positives": patent_positives,
        "cpc_positives": list(cpc_positives)
    }

# Step 3: Save to JSON
with open('ground_truth_2019_cpc.json', 'w') as f:
    json.dump(ground_truth, f, indent=2)

# Optional: Print some statistics about the ground truth
print(f"Created ground truth for {len(ground_truth)} figures")
if ground_truth:
    sample_figure = next(iter(ground_truth))
    print(f"Sample entry for {sample_figure}:")
    print(f"  Patent positives: {len(ground_truth[sample_figure]['patent_positives'])}")
    print(f"  CPC positives: {len(ground_truth[sample_figure]['cpc_positives'])}")

100%|██████████| 95299/95299 [00:06<00:00, 15174.33it/s]


Created ground truth for 1939 figures
Sample entry for USD0836880-20190101-D00001_1.png:
  Patent positives: 6
  CPC positives: 975


In [13]:
medium_cpc_cols = list(range(num_figures + num_patents, num_figures + num_patents + num_medium_cpc))

In [14]:
medium_cpc_cols

[34387,
 34388,
 34389,
 34390,
 34391,
 34392,
 34393,
 34394,
 34395,
 34396,
 34397,
 34398,
 34399,
 34400,
 34401,
 34402,
 34403,
 34404,
 34405,
 34406,
 34407,
 34408,
 34409,
 34410,
 34411,
 34412,
 34413,
 34414,
 34415,
 34416,
 34417,
 34418,
 34419,
 34420,
 34421,
 34422,
 34423,
 34424,
 34425,
 34426,
 34427,
 34428,
 34429,
 34430,
 34431,
 34432,
 34433,
 34434,
 34435,
 34436,
 34437,
 34438,
 34439,
 34440,
 34441,
 34442,
 34443,
 34444,
 34445,
 34446,
 34447,
 34448,
 34449,
 34450,
 34451,
 34452,
 34453,
 34454,
 34455,
 34456,
 34457,
 34458,
 34459,
 34460,
 34461,
 34462,
 34463,
 34464,
 34465,
 34466,
 34467,
 34468,
 34469,
 34470,
 34471,
 34472,
 34473,
 34474,
 34475,
 34476,
 34477,
 34478,
 34479,
 34480,
 34481,
 34482,
 34483,
 34484,
 34485,
 34486,
 34487,
 34488,
 34489,
 34490,
 34491,
 34492,
 34493,
 34494,
 34495,
 34496,
 34497,
 34498,
 34499,
 34500,
 34501,
 34502,
 34503,
 34504,
 34505,
 34506,
 34507,
 34508,
 34509,
 34510,
 34511,
